# Before your begin

1. Setup **gcloud** by following this guide [here](https://29022131.atlassian.net/wiki/spaces/DP/pages/1006174505/JupyterHub+-+End-user+Guide#JupyterHub-End-userGuide-GCloudsetup).
2. Setup **github** by following this guide [here](https://29022131.atlassian.net/wiki/spaces/DP/pages/1006174505/JupyterHub+-+End-user+Guide#JupyterHub-End-userGuide-Githubsetup).

# Export BQ data to GCS to Pandas Dataframe

Usually, a convenient way to convert BQ query result to pandas dataframe is to use to_dataframe() method. However, if your data is large, it will become significantly slower.
One way to tackle this problem is to export the query result to another BQ temp table, then export this table to GCS in CSV format, then load it to pandas dataframe.

In [2]:
from google.cloud import storage
from google.cloud import bigquery
import pandas as pd

#### Initiate bq client

In [3]:
client = bigquery.Client(project="YOUR-PROJECT-ID")

/home/jovyan/config/.conda/envs/andri/lib/python3.7/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


#### Sample query

In [4]:
query = (
        "SELECT "
        "trip_distance, fare_amount "
        "FROM "
        "`nyc-tlc.yellow.trips` "
        "WHERE "
        "trip_distance > 0 "
        "AND passenger_count > 0 "
        "AND fare_amount >= 2.5 "
        "AND pickup_longitude > -78 "
        "AND pickup_longitude < -70 "
        "AND dropoff_longitude > -78 "
        "AND dropoff_longitude < -70 "
        "AND pickup_latitude > 37 "
        "AND pickup_latitude < 45 "
        "AND dropoff_latitude > 37 "
        "AND dropoff_latitude < 45 "
        "AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 5000) = 1"
    )

#### Initiate bq job config

In [5]:
dataset_name = "YOUR-DATASET-NAME"

In [6]:
table_name = "YOUR-TEMP-TABLE-NAME"

In [7]:
job_config = bigquery.QueryJobConfig()
table_ref = client.dataset(dataset_name).table(table_name)
job_config.destination = table_ref
# by default it will truncate the table if exists
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

#### Execute the query and save the result to another BQ temp table (configured in the job_config above)

In [8]:
query_job = client.query(
    query,
    location="US",
    job_config=job_config)

#### Extract the table to a CSV file in GCS

In [9]:
destination_uri = "gs://tvlk-datappos-andri/bqtocsv/bqdata.csv"

In [10]:
extract_job = client.extract_table(
    table_ref,
    destination_uri,
    location='US')

#### Load CSV file in GCS to Pandas Dataframe

In [11]:
!pip install gcsfs

In [12]:
df = pd.read_csv(destination_uri)

/home/jovyan/config/.conda/envs/andri/lib/python3.7/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [13]:
df

,trip_distance,fare_amount
0,2,2.5
1,2,2.5
2,2,2.5
3,2,2.5
4,2,2.5
...,...,...
220852,0.58,4
220853,0.58,4
220854,0.58,4
220855,0.58,4


#### Deleting BQ temp table and GCS file

In [14]:
client.delete_table(table_ref)

In [15]:
storage_client = storage.Client()
bucket = storage_client.get_bucket("tvlk-datappos-andri")
blob = bucket.blob("bqtocsv/bqdata.csv")

blob.delete()

/home/jovyan/config/.conda/envs/andri/lib/python3.7/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/home/jovyan/config/.conda/envs/andri/lib/python3.7/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see 